**NOAA CalCOFI Genomics (NCOG) MetaT - Filtering ORFs belonging to Mixoplankton**

Definition of mixoplankton, [Mitra et al 2023](https://onlinelibrary.wiley.com/doi/10.1111/jeu.12972?s=03): eukaryotes that can perform
- photoautotrophy through constitutive (native to cell) photosynthetic machinery or machinery acquired from prey 
- osmoheterotrophy (taking in nutrients dissolved solution including necessary carbon)
- phagotrophy (taking in machinery and material from prey cells, by variety of feeding methods-- engulfing whole cells, sticking a feeding tube into them, lysing them with toxic metabolites)

Input files for analysis: 
1. pr2_version_5.0.0_taxonomy.tsv - Nine domain system and names for all 53k+ species represented in the PR2 database. Can use for taking taxonomic names from the other files and converting them to standardized nine-name taxonomy objects to check matching/equality.
- also contains a "mixoplankton" column. use this to filter out all non-mixos.

2. annotation_all.tab - Pratap Venepally's annotation of ORF gene function and taxonomy. Important column: LPI_taxonomy.

**Which file to use as input??**

These are different files with different sizes. Do they also have different contig annotations?
- /tscc/projects/ps-allenlab/archdata/pratap/NCOG/PolyA_rnaseq/RAP_analysis_042022/annotation_042022/annotation_all.tab (4G)
- /tscc/projects/ps-allenlab/archdata/pratap/NCOG/NCOG_RNASeq_merged_2014-2018-2019-2020_assemblies/merge_2014-18_2019_2020_assemblies/annotation_all.tab (15G)

The first file is for polyA RNA only, the latter file is total RNA. These correspond to 
- archdata/zfussy/RAP_runs/NCOG_totalRNA/annotation_all.tab (15G)
- archdata/zfussy/RAP_runs/NCOG_polyA/annotation_042022/annotation_all.tab (4G)

So are the annotations the same between each category? Yes (diff on the files producs below confirms)

`awk '{ print $1 }' /tscc/projects/ps-allenlab/archdata/pratap/NCOG/PolyA_rnaseq/RAP_analysis_042022/annotation_042022/annotation_all.tab > _contignames_pratap_polyA`

`awk '{ print $1 }' /tscc/projects/ps-allenlab/archdata/pratap/NCOG/NCOG_RNASeq_merged_2014-2018-2019-2020_assemblies/merge_2014-18_2019_2020_assemblies/annotation_all.tab > _contignames_pratap_total`

`awk '{ print $1 }' /tscc/projects/ps-allenlab/archdata/zfussy/RAP_runs/NCOG_polyA/annotation_042022/annotation_all.tab > _contignames_zoltan_polyA`

`awk '{ print $1 }' /tscc/projects/ps-allenlab/archdata/zfussy/RAP_runs/NCOG_totalRNA/annotation_all.tab > _contignames_zoltan_total`

Why are they different across categories; if so, why (is it because it maps only slightly different places during each search)

**Don't know! but now can return to notebook 6 and use the new annotations, if it's the same abundances nothing else will have changed (next time can run the aldex analysis on polyA only)**

In [1]:
import pandas as pd
contig_names_polyA = pd.read_csv("_contignames_pratap_polyA")
contig_names_total = pd.read_csv("_contignames_pratap_total")
contig_names_polyA

,orf_id
0,Spike1
1,Spike8
2,contig_1000000_1_675_-
3,contig_1000001_3_1871_-
4,contig_1000002_1_828_+
...,...
5043563,contig_999996_3_278_-
5043564,contig_99999_2_418_-
5043565,contig_99999_895_1068_+
5043566,contig_99_2_127_+


In [8]:
contig_names_total[contig_names_total["orf_id"].str.contains("contig_999990")]

,orf_id
9346245,NCOG_2014-18.contig_999990_1174_1953_+
9346246,NCOG_2014-18.contig_999990_1_1104_-
11841523,NCOG_2020.contig_999990_1_198_-
11841524,NCOG_2020.contig_999990_208_441_+


In [9]:
contig_names_polyA[contig_names_polyA["orf_id"].str.contains("contig_999990")]

,orf_id
5043555,contig_999990_165_239_+
5043556,contig_999990_1_138_+


(233, 779)

**Preprocessing**

data frames and helper functions for accessing data for figures & tests

field format for NCOG data, [NCOG line/station key](https://calcofi.org/sampling-info/station-positions/): [On daily noon stations and cardinal stations (lines 80 and 90; see map), RNA and DNA seawater samples are collected from 10m and the depth of the chlorophyll maximum (i.e. the depth of highest chlorophyll-a concentration, which varies from station to station). RNA is also sampled from 170 m and 515 m where possible.](https://calcofi.org/data/marine-ecosystem-data/e-dna/)
- X201402_086.7_033.0_10
    - X\[YYYY\]\[MM\]_\[line number\]_\[station number\]_\[depth, either 10m or deep Chl maximum\]
 
In the NCOG dataset there are 34,498 ASVs found across 1195 samples (different times, locations, depths) and classified by 4 different classifiers.

In [34]:
import pandas as pd
import numpy as np

In [ ]:
"""reading in the pr2 taxonomies - this file has no index column"""

pr2_taxonomies = pd.read_csv("pr2_version_5.0.0_taxonomy.tsv", sep = "\t", header = 0)

In [ ]:
#might throw in the MDB species which pr2 doesn't include for whatever reason.
mixo_mask = [type(item) == str for item in pr2_taxonomies["mixoplankton"]]
#"domain" is obvious, also want to get rid of plastid sequences...? are they polyA
mixo_species = pr2_taxonomies.loc[mixo_mask, "species"]
mixo_species = mixo_species[ [item.find(":plas") == -1 for item in mixo_species] ]

mixo_species.to_csv("01_out_pr2_mixo_species.tsv", sep="\t", header=False, index=False)
#taxonomic_levels = pr2_taxonomies.columns[1:9]
#mixo_taxonomies = pr2_taxonomies.loc[mixo_mask, taxonomic_levels]

In [ ]:
from search_mixo_species import search_species #so that this imports the latest version of "mixo_species"

**Match species to PR2 mixodatabase - shouldn't accept any matches that don't go down to species level.**

In [27]:
mixo_species[(mixo_species.str.contains("Durinskia"))]
not mixo_species[(mixo_species.str.contains("Geminigera"))].empty
"Durinskia_capensis" in mixo_species.values

True

In [4]:
#since phylodb lpi taxonomy seems to follow pr2 conventions...
def search_taxonomy(taxonomy):
    taxons = taxonomy.split(";")
    most_specific_name = taxons[-1]
    #start with the most specific name and work backward
    for level in taxonomic_levels[::-1]:
        if most_specific_name in mixo_taxonomies[level]:
            return True

#format is nonstandard. pr2 can only match the first two terms anyways. make sure they're cleaned of commas, etc.
        
#Eukaryota;Archaeplastida;Chlorophyta;Nephroselmidophyceae;Nephroselmidophyceae_X;Nephroselmidaceae;Nephroselmis;Nephroselmis pyriformis, Strain CCMP717
#Eukaryota;Opisthokonta;Metazoa;Arthropoda;Hexapoda;Insecta;Acyrthosiphon;Acyrthosiphon pisum
#Eukaryota;Stramenopiles;Stramenopiles_X;Bacillariophyta;Bacillariophyta_X;Polar-centric-Mediophyceae;Thalassiosira;Thalassiosira oceanica CCMP1005


In [35]:
#search_species("Eukaryota;Archaeplastida;Chlorophyta;Nephroselmidophyceae;Nephroselmidophyceae_X;Nephroselmidaceae;Nephroselmis;Nephroselmis pyriformis, Strain CCMP717", debug = True)
#search_species("Eukaryota;Stramenopiles;Stramenopiles_X;Bacillariophyta;Bacillariophyta_X;Polar-centric-Mediophyceae;Thalassiosira;Thalassiosira oceanica CCMP1005", debug = True)
#search_species("Eukaryota;Opisthokonta;Metazoa;Arthropoda;Hexapoda;Insecta;Acyrthosiphon;Acyrthosiphon pisum", debug = True)
#search_species("Eukaryota;Hacrobia;Haptophyta;Prymnesiophyceae;Isochrysidales;Noelaerhabdaceae;Emiliania;Emiliania huxleyi 374", debug=True)
#search_species("Eukaryota;Hacrobia;Haptophyta;Prymnesiophyceae;Isochrysidales;Noelaerhabdaceae;Emiliania;Emiliania huxleyi CCMP1516", debug=True)
#search_species("Eukaryota;Hacrobia;Haptophyta;Prymnesiophyceae;Isochrysidales;Noelaerhabdaceae;Emiliania;Emiliania huxleyi CCMP370", debug=True)
search_species("Eukaryota;Hacrobia;Haptophyta;Prymnesiophyceae;Phaeocystales;Phaeocystaceae;Phaeocystis;Phaeocystis antarctica, Strain CCMP1374", debug=True)

Phaeocystis_antarctica


False

In [6]:
mixo_species[mixo_species.str.contains("Phaeocystis")]

9410    Phaeocystis_globosa
Name: species, dtype: object

In [7]:
with open("annotation_all.filtered.orfhits.tab", "r") as file:
    print(file.readline().split("\t")[-2:])

['NR0557', 'NR0558\n']
['See\n', 'ya\n']


In [36]:
#not that many mixo sequences, might be able to keep the orf_ids in memory. make them a set then use them to 
#filter rows of tpm file.

#modify this code to generate annotation file: this doesn't have to have the same row ordering as the counts file.

mixo_orfs = set()
with open("annotation_all.tab", "r") as infile:
    with open("01_out_mixo_orfs_annotations.tsv", "w") as outfile:
        #transfer the first line's column labels to outfile
        col_labels = infile.readline().split("\t")
        annotation_labels = col_labels[:37]
        assert(annotation_labels[36] == "LPI_taxonomy")
        outfile.write("\t".join(annotation_labels) + "\n")
        
        #moving on to data rows
        for line in infile: 
            terms = line.split("\t") #includes annotations and read counts
            annotation_terms = terms[:37] #drop read counts
            orf_id = annotation_terms[0]
            lpi_taxonomy = annotation_terms[36]
            #if lpi_taxonomy == "" or "Spike" in orf_id:
            if "Eukaryota" not in lpi_taxonomy:
                continue #no point looking here: either taxonomy unresolved or not eukaryotic
            #if taxonomy is eukaryotic:
            if search_species(lpi_taxonomy):
                #what's the most useful output? a one-line long string that R script can use to index?
                #or a long column that strsplit can make into a vector or list of strings?
                mixo_orfs.add(orf_id)
                #count += 1
                outfile.write("\t".join(annotation_terms) + "\n")

In [37]:
len(mixo_orfs) #342726 expected

324726

In [ ]:
assert(False)

In [ ]:
#so now it needs them to be integers
#Error in aldex.clr.function(reads, conds, mc.samples, denom, verbose,  : 
#  not all reads are integers
with open("annotation_all.filtered.orfhits.tab") as infile:
    with open("01_out_mixo_orfs_rawcounts.tsv", "w") as outfile:
        #print the first line
        outfile.write(infile.readline())
        
        for line in infile:
            #['"Spike8"'], need to strip the quotemarks from both sides
            orf_id = line.split("\t")[0].strip("\"")
            #print([orf_id])
            if orf_id in mixo_orfs:
                outfile.write(line)
                
#filtering probably leads to less ASVs, i think it's fine. run this now and try w/ unclustered asvs later if needed.

In [ ]:
%%bash
wc -l 01_out_mixo_orfs_rawcounts.tsv